# Review analysis

In [1181]:
# Load the data
from pathlib import Path
import pandas as pd
import numpy as np

data_csv = Path("reviews.csv")

if not data_csv.exists():  # make sure the file exists, otherwise compile it
    from result_compiler import main as data_compiler
    data_compiler()

df = pd.read_csv(data_csv)

# Sort the sources in the DataFrame as AAAI, IJCAI then ICML, NeurIPS and JAIR, JMLR
source_order = ["AAAI", "IJCAI", "ICML", "NeurIPS", "JAIR", "JMLR"]
df["source"] = pd.Categorical(df["source"], source_order)
# Drop the index column
df.drop("index", axis=1, inplace=True)

# Import the analysis/visualisation libraries
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.kaleido.scope.mathjax = None

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)  

# Define default plot layout
review_categories = ["Implementation", "Data", "Configuration", "Experimental Procedure", "Expertise"]
width = 450
height = 200
layout = dict(
    boxmode="group",
    width=width * (1/0.6),
    height=height * (1/0.6),
    font=dict(
        family="serif",
        size=15,
    ),
    margin=dict(
        t=10,
        b=10,
        l=10,
        r=10,
    ),
    xaxis=dict(
        tickangle=0,
    ),
    yaxis=dict(
        title="Cost",
        range=[1, 10],
        dtick = 1,
    ),
    legend_title=None,
    #xaxis_title="Years",
    #plot_bgcolor="rgba(0, 0, 0, 0)",
    #paper_bgcolor="rgba(0, 0, 0, 0)",
    #legend=dict(orientation="h", yanchor="top", y=1.1, xanchor="center", x=0.5),
    legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
)

In [1182]:
# Analysis functions

def display_table_statistics(df: pd.DataFrame, columns: list) -> None:
    """Create a table from the selected data and print it."""
    df2 = df.drop([c for c in df.columns if c not in columns], axis=1).T
    dropping = df2.columns
    mean = df2.mean(axis=1)
    std = df2.std(axis=1)
    q1 = df2.quantile(q=0.25, axis=1)
    q2 = df2.quantile(q=0.5, axis=1)
    q3 = df2.quantile(q=0.75, axis=1)
    variance = df2.var(axis=1)
    df2 = df2.drop(dropping, axis=1)
    df2["mean"] = mean
    df2["std"] = std
    df2["q1"] = q1
    df2["q2"] = q2
    df2["q3"] = q3
    df2["variance"] = variance
    display(df2.round(4))

def display_box_plot(df: pd.DataFrame, layout: dict, title: str, labels: list) -> None:
    """Create a boxplot from the selected data and print it."""
    fig = px.box(df, y=labels, color="year", title=title, labels=labels)
    fig.update_layout(layout)
    fig.show()

def display_bar_plot(df: pd.DataFrame, layout: dict, title: str, labels: list) -> None:
    """Create a barplot from the selected data and print it."""
    fig = px.bar(df, x="year", y=labels, labels=labels)
    layout_adapt = layout.copy()
    layout_adapt["barmode"] = "group"
    layout_adapt["yaxis"] = dict(range=[1, 350], dtick=50, title=title)
    fig.update_layout(layout_adapt)
    fig.show()

def display_correlation_matrix(df: pd.DataFrame, filter=None, threshold = 0.25) -> None:
    """Create a correlation matrix from the selected data and print it.

    Args:
        df: The dataframe to create the correlation matrix from.
        filter: Columns to filter
        threshold: The threshold to apply to the correlation matrix for visualisation.
    """
    def dfstyle(val):
        map = []
        for v in val:
            if not isinstance(v, float) or abs(v) > threshold:
                map.append("font-weight: bold")
            else:
                map.append("font-weight: normal")
        return map

    # We select Kendall because it makes no assumptions about the data distributions,
    # and its robust to outliers and non-linearity.
    correlation = df.corr(method="kendall", numeric_only=True)
    if filter:
        correlation = correlation.drop(filter, axis=1)
    correlation.mask(correlation == 1.0, inplace=True)
    display(correlation.style.apply(dfstyle))

def display_correlation_heatmap(df: pd.DataFrame, title: str, show_title_plot=False, filter=None) -> None:
    """Create a correlation heatmap from the selected data and print it."""
    corr_df = df.corr(method="kendall", numeric_only=True).round(2)
    if filter:
        corr_df = corr_df.drop(filter, axis=1).drop(filter, axis=0)
    # Mark the 'significant' results bold face?
    fig = px.imshow(corr_df, text_auto=True, zmin=-1, zmax=1, color_continuous_scale="Viridis", title=title if show_title_plot else None)
    fig.write_image(f"latex/plots/{title}.pdf")
    fig.show()

def display_covariance_heatmap(df: pd.DataFrame, title: str, filter=None) -> None:
    """Create a covariance heatmap from the selected data and print it."""
    cov_df = df.cov(numeric_only=True).round(2)
    if filter:
        cov_df = cov_df.drop(filter, axis=1).drop(filter, axis=0)
    np.fill_diagonal(cov_df.values, 0)  # Mask the variance
    fig = px.imshow(cov_df, text_auto=True, color_continuous_scale="Viridis", title=title)
    fig.show()

## General Metrics

In [1183]:
# Create a general dataset statistics table
import importlib
import latex
importlib.reload(latex)
header = source_order
years = list(df["year"].unique())

values = []
for year in years:
    values.append([])
    for source in header:
        subset = df[(df["source"] == source) & (df["year"] == year)]
        values[-1].append(f"{len(subset[subset["theoretical"] == False])} ({len(subset[subset["theoretical"] == True])})")

# Add total row
values.append([])
for source in header:
    subset = df[(df["source"] == source)]
    values[-1].append(f"{len(subset[subset["theoretical"] == False])} ({len(subset[subset["theoretical"] == True])})")
gen_stats = latex.latex_table(columns=header, rows=years + ["Total"], values=values, label="tab:data_summary", column_char="r",
                              caption="A summary of the collected data, where the first number represents the number of applicable reviews "
                                      "and the number of excluded (theoretical) papers is represented in brackets. "
                                      f"We collected {len(df[df["theoretical"] == False])} applicable reviews out of {len(df)} total papers reviewed.")
latex.compile_latex(gen_stats, "tables/general_statistics")
# Measure the amount of theoretical papers that we will have to discount
theoretical_papers = df["theoretical"].sum()
print(f"Total number of theoretical papers: {theoretical_papers}. Percentage: {(theoretical_papers / len(df) * 100).round(4)}%")

df = df[df["theoretical"] == False]
df.drop(["theoretical"], axis=1, inplace=True)
print(f"Total number of applicable papers: {len(df)}")

# Measure the amount of papers that have implementation URLS
print(f"Total number of implementation URLs: {df["implementation_url"].sum()}. Percentage: {(df["implementation_url"].sum() / len(df) * 100).round(4)}%")

# Measure the amount of data sets used and how many were public vs private
avg = (df["public_datasets"].sum() / df["total_datasets"].sum() * 100).round(2)
title = f"Public data sets vs Total Data sets ({avg}%)"
plot = px.bar(df, x="public_datasets", y="total_datasets", title=title)
plot.show()

df["public_data_fraction"] = df["public_datasets"] / df["total_datasets"]
print("Number of papers / percentage that only use private data:", len(df[df["public_datasets"] == 0]), round(len(df[df["public_datasets"] == 0]) / len(df) * 100, 2))
df.drop(["public_datasets", "total_datasets"], axis=1, inplace=True)


# Calculate the average rating for each paper (Excluding expertise)
df["Average"] = df[review_categories[0:-1]].mean(axis=1)

Total number of theoretical papers: 134. Percentage: 14.7091%
Total number of applicable papers: 777
Total number of implementation URLs: 477. Percentage: 61.39%


Number of papers / percentage that only use private data: 31 3.99


In [1184]:
# Compare award papers per source and check if they are in or out of distribution
import itertools
importlib.reload(latex)
import scipy
ks_table = [[], []]

conference_source_order = ["AAAI", "IJCAI", "ICML", "NeurIPS"]
years = df["year"].unique()

all_combinations = [(source, year) for source, year in itertools.product(source_order, years)]
all_combination_title = [f"{source}, {year}" for source, year in all_combinations]
import math
for source in conference_source_order:
    kstest_res = scipy.stats.kstest(df[(df["source"] == source) & (df["awards"].isna())]["Average"].to_numpy(),
                                    df[(df["source"] == source) & (df["awards"].notna())]["Average"].to_numpy())
    result = f"{kstest_res.statistic:.2f} ({kstest_res.pvalue:.2f})"
    # Formulae from the wikiu for 'large samples' to reject the null hypothesis at alpha, where c(alpha) == c(0.05) == 1.358
    # sdf = df[df["source"] == source]
    # n = len(sdf) - len(sdf[sdf["awards"].isna()])
    # m = len(sdf[sdf["awards"].isna()])
    # wiki_value = 1.358 * math.sqrt((n + m ) / (n * m))
    # print(f"Wiki test: {kstest_res.statistic} > {wiki_value}", kstest_res.statistic > wiki_value)
    print(source, result)

    if kstest_res.pvalue < 0.05:
        result = "\\textbf{" + f"{kstest_res.statistic:.2f}" + "} " + f"({kstest_res.pvalue:.2f})"
    ks_table[0].append(result)

for source in conference_source_order:
    ttest_res = scipy.stats.ttest_ind(
        df[(df["source"] == source) & (df["awards"].notna())]["Average"].to_numpy(),
        df[(df["source"] == source) & (df["awards"].isna())]["Average"].to_numpy(), alternative="less")
    result = f"{ttest_res.statistic:.2f} ({ttest_res.pvalue:.2f})"
    if ttest_res.pvalue < 0.05:
        result = "\\textbf{" + f"{ttest_res.statistic:.2f}" + "} " + f"({ttest_res.pvalue:.2f})"
    ks_table[1].append(result)
    print(source, result)

ks_table = latex.latex_table(conference_source_order, ["KS-Test result", "T-Test result"], ks_table, column_char="r",
                             caption="Kolmogorov–Smirnov test and T-Test per source between papers with and without awards. The P-values are in brackets. The statistically significant results are highlighted in \\textbf{bold}.",
                             label="tab:ks_test_sources_awards")

latex.compile_latex(ks_table, "tables/ks_test_awards")

# Remove awards from population
df = df[df["awards"].isna()]

AAAI 0.34 (0.64)
IJCAI 0.48 (0.06)
ICML 0.26 (0.13)
NeurIPS 0.09 (1.00)
AAAI 0.22 (0.59)
IJCAI \textbf{-2.22} (0.01)
ICML 2.17 (0.98)
NeurIPS 0.16 (0.56)


In [1185]:
bins = pd.IntervalIndex.from_tuples([(0.9999, 3), (3.00001, 6), (6.00001, 10)])
stratum_labels = ["Low", "Medium", "High"]
cut = pd.cut(df["Average"], bins, labels=stratum_labels)
cut = cut.cat.rename_categories(stratum_labels)
df["stratum"] = cut

In [1186]:
# Put the stratum count in a latex table
values = []
total = [0, 0, 0]
for source in source_order:
    source_count = df[df["source"] == source]["stratum"].value_counts()
    low_count, medium_count, high_count = source_count["Low"], source_count["Medium"], source_count["High"]
    total[0] += low_count
    total[1] += medium_count
    total[2] += high_count
    values.append([low_count, medium_count, high_count])
    print(f"{source}:", source_count["Low"], source_count["Medium"], source_count["High"])

values.append(total)

import importlib
import latex
importlib.reload(latex)
stratum_table = latex.latex_table(columns=stratum_labels, rows=source_order + ["Total"], values=values,
                                  label="tab:stratum_count",
                                  caption="The number of papers in each stratum, separated by source.")
latex.compile_latex(stratum_table, "tables/stratum_count")


AAAI: 58 68 12
IJCAI: 60 54 9
ICML: 78 50 2
NeurIPS: 72 45 4
JAIR: 57 39 4
JMLR: 53 54 7


In [1187]:
df.sort_values(["stratum"], ascending=True, inplace=True)

# Create a stacked barplot for each source based on the bins per dimension
fig = px.histogram(df, x="source", barmode="group", color="stratum", labels={
             "count": "Count",
             "source": "",},
             category_orders={"source": source_order}
             )
layout_adapt = layout.copy()
layout_adapt["yaxis"] = dict(range=[1, 100])
layout_adapt["margin"]["b"] = 0
fig.update_layout(layout_adapt)
fig.update_layout(xaxis_title=None)
#bar_percentages = [[1,2,3], [4,5,6], [7,8,9], [10,11,12], [13,14,15], [16,17,18]]
bar_percentages = [[1,2,3,4,5,6], [7,8,9,10,11,12], [13,14,15,16,17,18]]
bar_percentages = [[],[],[]]
for index, label in enumerate(stratum_labels):
    for source in source_order:
        total = len(df[df["source"] == source])
        fraction = len(df[(df["source"] == source) & (df["stratum"] == label)]) / total
        bar_percentages[index].append(round(fraction, 2))
# Place percentages on histogram
for i, t in enumerate(bar_percentages):
    fig.data[i].text = t
    fig.data[i].textposition = 'outside'
    fig.data[i].textfont = {"size": 10}
fig.write_image("latex/plots/stratum_count.pdf")
fig.show()
import plotly
plotly.graph_objs._histogram.Histogram


plotly.graph_objs._histogram.Histogram

In [1188]:
# CDF plot as well to provide more information than Histogram
fig = px.ecdf(df, x="Average", color="source", ecdfnorm="percent")
layout_adapt = layout.copy()
del layout_adapt["xaxis"]
del layout_adapt["yaxis"]
layout_adapt["yaxis_title"] = "Percentage of Papers"
layout_adapt["xaxis_title"] = "Average Cost"
layout_adapt["xaxis_range"] = [1,10]
#layout_adapt["yaxis"]["range"] = [0,1]
fig.update_layout(layout_adapt)
#fig.update_xaxes(dtick=0.1)
fig.write_image("latex/plots/CDF_average_cost.pdf")
fig.show()

In [1189]:
print(f"Q1: Bottom 33% of ratings: {df['Average'].quantile(.33)}") 
print(f"Q2: Bottom 66% of ratings: {df['Average'].quantile(.66)}")
print(f"Average Rating: {round(df['Average'].mean(), 2)}")
print((df["Average"] < df["Average"].quantile(.33)).sum(), (df["Average"] > df["Average"].quantile(.66)).sum())


Q1: Bottom 33% of ratings: 2.25
Q2: Bottom 66% of ratings: 4.0
Average Rating: 3.29
215 228


In [1190]:
# Random statistic for fun
def average_title_length(df: pd.DataFrame, source: str, year: int) -> None:
    return df.query(f"source == '{source}' and year == {year}")["title"].apply(len).mean().__round__(2)
print("AAAI average title lengths over 3 years:",
      average_title_length(df, "AAAI", 2022),
      average_title_length(df, "AAAI", 2023),
      average_title_length(df, "AAAI", 2024))
print("ICML average title lengths over 3 years:",
      average_title_length(df, "ICML", 2022),
      average_title_length(df, "ICML", 2023),
      average_title_length(df, "ICML", 2024))
print("IJCAI average title lengths over 3 years:",
      average_title_length(df, "IJCAI", 2022),
      average_title_length(df, "IJCAI", 2023),
      average_title_length(df, "IJCAI", 2024))
print("JAIR average title lengths over 3 years:",
      average_title_length(df, "JAIR", 2022),
      average_title_length(df, "JAIR", 2023),
      average_title_length(df, "JAIR", 2024))
print("JMLR average title lengths over 3 years:",
      average_title_length(df, "JMLR", 2022),
      average_title_length(df, "JMLR", 2023),
      average_title_length(df, "JMLR", 2024))
print("NeurIPS average title lengths over 3 years:",
      average_title_length(df, "NeurIPS", 2022),
      average_title_length(df, "NeurIPS", 2023),
      average_title_length(df, "NeurIPS", 2024))


AAAI average title lengths over 3 years: 76.18 74.6 82.33
ICML average title lengths over 3 years: 67.62 72.87 74.6
IJCAI average title lengths over 3 years: 74.0 73.9 78.07
JAIR average title lengths over 3 years: 71.49 74.86 71.94
JMLR average title lengths over 3 years: 73.15 68.21 71.39
NeurIPS average title lengths over 3 years: 63.42 75.02 73.47


In [1191]:
# Average rating by source + standard deviation per dimension
import importlib
import latex
importlib.reload(latex)
statistics_table = df.drop(["title", "authors", "keywords", "pdf_path", "awards", "implementation_url", "year"], axis=1)
mean = statistics_table.drop("public_data_fraction", axis=1).groupby(["source"]).mean(numeric_only=True)
std = statistics_table.drop("public_data_fraction", axis=1).groupby(["source"]).std(numeric_only=True)

values = []
for row in mean.index:
    values.append([])
    for column in mean.columns:
        values[-1].append(f"${mean[column][row]:.2f} \\pm {std[column][row]:.2f}$")

column_index = []
for index, column in enumerate(mean.columns):
    if column == "Experimental Procedure":
        column_index.append("Exp. Procedure")
    else:
        column_index.append(column)
l_table = latex.latex_table(column_index, mean.index.unique(), values, column_char="r",
                            caption="Average cost with one standard deviation per source and dimension.",
                            label="tab:average_cost_source")
latex.compile_latex(l_table, "tables/average_cost_source")


/var/folders/78/9cl0kydx0g3cdzjyk28yzr7w0000gn/T/ipykernel_33255/3296503233.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/78/9cl0kydx0g3cdzjyk28yzr7w0000gn/T/ipykernel_33255/3296503233.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [1192]:
# Average rating by year and source + standard deviation
import importlib
import latex
importlib.reload(latex)
statistics_table = df.drop(["title", "authors", "keywords", "pdf_path", "awards", "implementation_url"], axis=1)
mean = statistics_table.drop("public_data_fraction", axis=1).groupby(["source", "year"]).mean(numeric_only=True)
std = statistics_table.drop("public_data_fraction", axis=1).groupby(["source", "year"]).std(numeric_only=True)

values = []
for row in mean.index:
    values.append([])
    for column in mean.columns:
        values[-1].append(f"${mean[column][row]:.2f} \\pm {std[column][row]:.2f}$")

row_index = []
for index, column in enumerate(mean.index):
    if index > 0 and mean.index[index - 1][0] == column[0]:
        row_index.append(["", column[1]])
    else:
        row_index.append([column[0], column[1]])

column_index = []
for index, column in enumerate(mean.columns):
    if column == "Experimental Procedure":
        column_index.append("Exp. Proc.")
    else:
        column_index.append(column)
l_table = latex.latex_table(column_index, row_index, values, column_char="r",
                            caption="Average cost with one standard deviation per source, year and dimension.",
                            label="tab:average_cost_source_year")
latex.compile_latex(l_table, "tables/average_cost_source_year")


/var/folders/78/9cl0kydx0g3cdzjyk28yzr7w0000gn/T/ipykernel_33255/569327187.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/78/9cl0kydx0g3cdzjyk28yzr7w0000gn/T/ipykernel_33255/569327187.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [1193]:
# Create a table for the implementation url % and the publicness of data
values = [[],[]]
for source in source_order:
    url_percentage = df[df["source"] == source]["implementation_url"].mean().round(4) * 100
    values[0].append(f"${url_percentage} \\%$")
    data_percentage = df[df["source"] == source]["public_data_fraction"].mean().round(4) * 100
    values[1].append(f"${data_percentage} \\%$")

values[0].append(f"${df["implementation_url"].mean().round(4) * 100} \\%$")
values[1].append(f"${df["public_data_fraction"].mean().round(4) * 100} \\%$")

import latex
metric_tab = latex.latex_table(source_order + ["Average"], ["Implementation URL", "Public Data"], values=values,
                               label="tab:data_metrics",
                               caption="The measured rates of whether implementation links were specified and how many of the used datasets were public, per source.")
latex.compile_latex(metric_tab, "tables/data_metrics")

In [1194]:
# Compare all sources if they are significantly different according to KS
import scipy
importlib.reload(latex)
ks_table = []
for source in source_order:
    ks_table.append([])
    for other_source in source_order:
        if source == other_source:
            ks_table[-1].append("-")
            continue
        kstest_res = scipy.stats.kstest(df[df["source"] == source]["Average"].to_numpy(), df[df["source"] == other_source]["Average"].to_numpy())
        result = f"{kstest_res.statistic:.2f} ({kstest_res.pvalue:.2f})"
        if kstest_res.pvalue < 0.05:
            result = "\\textbf{" + f"{kstest_res.statistic:.2f}" + "} " + f"({kstest_res.pvalue:.2f})"
        ks_table[-1].append(result)

ks_table = latex.latex_table(source_order, source_order, ks_table, column_char="r",
                             caption="Kolmogorov–Smirnov test between sources. The statistically significant results are highlighted in \\textbf{bold}.",
                             label="tab:ks_test_sources")
latex.compile_latex(ks_table, "tables/ks_test_sources")


In [1195]:
# Compare all sources if they are significantly different according to T-test
import scipy
importlib.reload(latex)

ttest_table = []
for source in source_order:
    ttest_table.append([])
    for other_source in source_order:
        if source == other_source:
            ttest_table[-1].append("-")
            continue

        ttest_res = scipy.stats.ttest_ind(df[df["source"] == source]["Average"].to_numpy(), df[df["source"] == other_source]["Average"].to_numpy(), alternative="less")
        result = f"{ttest_res.statistic:.2f} ({ttest_res.pvalue:.2f})"
        if ttest_res.pvalue < 0.05:
            print(f"{source} is less than {other_source} by {result}")
            result = "\\textbf{" + f"{ttest_res.statistic:.2f}" + "} " + f"({ttest_res.pvalue:.2f})"
        ttest_table[-1].append(result)

ks_table = latex.latex_table(source_order, source_order, ttest_table, column_char="r",
                             caption="T-test between sources. The statistically significant results are highlighted in \\textbf{bold}.",
                             label="tab:ttest_sources")
latex.compile_latex(ks_table, "tables/ttest_sources")


ICML is less than AAAI by -3.98 (0.00)
ICML is less than IJCAI by -3.07 (0.00)
ICML is less than JMLR by -2.64 (0.00)
NeurIPS is less than AAAI by -2.52 (0.01)
NeurIPS is less than IJCAI by -1.70 (0.05)
JAIR is less than AAAI by -2.26 (0.01)


In [1196]:
display_correlation_heatmap(df.rename({"implementation_url": "URL", "public_data_fraction": "Public Data"}, axis=1), "Correlation Heatmap", filter=["Average", "year"])

In [1197]:
# Create a correlation heatmap per source as subplots
filter = ["year", "Average"]

# Arrange subfigures axis for paper
#enable_axis = [(True, False), (False, False), (True, False), (False, False), (True, True), (False, True)]
for index, source in enumerate(source_order):
    corr_df = df[df["source"] == source].corr(method="kendall", numeric_only=True).round(2)
    corr_df = corr_df.drop(filter, axis=1).drop(filter, axis=0)
    corr_df = corr_df.rename({"implementation_url": "URL", "public_data_fraction": "Public Data"}, axis=1)
    corr_df = corr_df.rename({"implementation_url": "URL", "public_data_fraction": "Public Data"}, axis=0)
    sub_fig = px.imshow(corr_df, text_auto=True, zmin=-1, zmax=1, color_continuous_scale="Viridis", title=f"{source}")
    #if not enable_axis[index][0]:
    #    sub_fig.update_yaxes(showticklabels=False)
    #if not enable_axis[index][1]:
    #    sub_fig.update_xaxes(showticklabels=False)
    sub_fig.write_image(f"latex/plots/{source}_correlation.pdf")
sub_fig.show()

## AAAI Analysis

In [1198]:
aaai_df = df[df["source"] == "AAAI"]

In [1199]:
display_table_statistics(aaai_df, review_categories)

,mean,std,q1,q2,q3,variance
Implementation,5.7029,3.5027,2.0,7.0,9.0,12.2688
Data,2.5580,1.6959,1.0,2.0,3.0,2.8762
Configuration,4.0290,2.4163,2.0,4.0,5.0,5.8386
Experimental Procedure,2.2391,1.4874,1.0,2.0,3.0,2.2125
Expertise,4.6232,1.9264,3.0,4.0,6.0,3.7110


In [1200]:
display_box_plot(aaai_df, layout, "AAAI Average rating", ["Average"])

In [1201]:
display_box_plot(aaai_df, layout, "AAAI reviews", review_categories)

In [1202]:
display_bar_plot(aaai_df, layout, "AAAI reviews", review_categories)

## ICML Analysis

In [ ]:
icml_df = df[df["source"] == "ICML"]

In [ ]:
display_box_plot(icml_df, layout, "ICML Average rating", ["Average"])

In [ ]:
display_box_plot(icml_df, layout, "ICML reviews", review_categories)

In [ ]:
display_correlation_heatmap(icml_df, "ICML Correlation Heatmap")

In [ ]:
display_covariance_heatmap(icml_df, "ICML Covariance Heatmap")

## IJCAI Analysis

In [ ]:
ijcai_df = df[df["source"] == "IJCAI"]

In [ ]:
display_box_plot(ijcai_df, layout, "IJCAI Average rating", ["Average"])

In [ ]:
display_box_plot(ijcai_df, layout, "IJCAI reviews", review_categories)

In [ ]:
display_correlation_heatmap(ijcai_df, "IJCAI Correlation Heatmap")

In [ ]:
display_covariance_heatmap(ijcai_df, "IJCAI Covariance Heatmap")

## NeurIPS Analysis

## JAIR Analysis

## JMLR Analysis

## Total Analysis

In [ ]:
# Do a Nemenyi test to find rankings between the sources